In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import math
import re
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from spellchecker import SpellChecker
from word2number import w2n

In [2]:
def is_number(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [21]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
spell = SpellChecker()

### Merge Duplicate data in train

In [22]:
def merge(grp):
    df = pd.DataFrame()
    if(grp.shape[0] > 1):
        for c in grp.columns:
            value_counts = grp[c].value_counts().index.astype(grp[c].dtypes)
            if value_counts.size > 1:
                print(grp, value_counts) #Error
            elif value_counts.size == 1:
                df[c] = value_counts[0]
            else:
                df[c] = None
    else:
        df = grp.head(1)
    return df
train = train.groupby("id").apply(merge)

In [23]:
ntrain = train.shape[0]
ntest = train.shape[0]
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['stroke_in_2018'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (43342, 13)


E:\Program\Anaconda\envs\NLP\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


### Data Cleaning

In [24]:
def is_job_status_and_living_area_reversed(x):
    return ((x["job_status"] != None and x["job_status"] in ("r", "c", "city", "remote", "remotee"))
             or (x["living_area"] != None and x["living_area"] in ("private_sector", "business_owner")))
def process_job_status(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x in ("private sector", "privattte", "private", "private_sector"):
        return "private_sector"
    elif x in ("government", "govt."):
        return "government"
    elif x in ("business_owner", "business owner", "biz"):
        return "business_owner"
    elif x in ("parental_leave", "parental leave"):
        return "parental_leave"
    else:
        return x
    
def process_living_area(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x == 'c':
        return 'city'
    elif x in ('r', 'remotee'):
        return 'remote'
    else:
        return x

def split_job_status_and_living_area(x):
    pair = x.lower().split("?") if x != None else [x, x]
    if len(pair) < 2:
        pair = [pair[0], None]
    return pair

def process_job_status_and_living_area(df):
    df["job_status"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[0])
    df["living_area"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[1])
    job_status = df.apply(lambda x: x["living_area"] if is_job_status_and_living_area_reversed(x) else x["job_status"], 1)
    living_area = df.apply(lambda x: x["job_status"] if is_job_status_and_living_area_reversed(x) else x["living_area"], 1)
    df["job_status"] = job_status.apply(lambda x: process_job_status(x))
    df["living_area"] = living_area.apply(lambda x: process_living_area(x))
    df.drop(columns='job_status and living_area',inplace=True)

In [25]:
process_job_status_and_living_area(all_data)
#process_job_status_and_living_area(test)

In [26]:
def process_smoker_status(x):
    if x == None:
        return None
    elif x.startswith("non"):
        return "non-smoker"
    elif x.startswith("quit"):
        return "quit"
    elif x.startswith("active"):
        return "active_smoker"
    else:
        return None
all_data["smoker_status"] = all_data["smoker_status"].astype(str).apply(process_smoker_status)
#test["smoker_status"] = test["smoker_status"].astype(str).apply(process_smoker_status)

In [27]:
def process_binary_col(df,columns):
    for col in columns:
        df[col] = pd.to_numeric(df[col],errors="coerce")
        df[col] = df[col].astype(int,errors='ignore')
        df[col] = df[col].apply(lambda x: x if x in [0,1] else None)

In [28]:
process_binary_col(all_data,["heart_condition_detected_2017","high_BP","married"])

Convert BMI to numeric

In [29]:
all_data["BMI"] = pd.to_numeric(all_data["BMI"],errors="coerce")

In [30]:
# Process gender into oneof "F", "M" and "OTHER"
def genderSpellingRewrite(gender_str):
    if not isinstance(gender_str, str):
        return None;
    uppered = gender_str.upper()
    # Repeated single occurence should be truncate.
    patternM = re.compile('[M]+$')
    if (patternM.match(uppered)):
        return "M"
    patternF = re.compile('[F]+$')
    if (patternF.match(uppered)):
        return "F"
    # Misspelling should be corrected and replaced.
    # TODO: Malle is not going to be corrected as Male.Need to update spell's known list.
    corrected = spell.correction(uppered).upper()
    if (corrected == "FEMALE"):
        return "F"
    if (corrected == "MALE"):
        return "M"
    if (corrected == "OTHER"):
        return "OTHER"
    return None

In [31]:
# Process human number word into number
def numberConversion(potential_number_word):
    # Correct any possible miss spelled number_word
    corrected_potential_word = spell.correction(potential_number_word)
    # check it it means number
    try:
      potential_num = w2n.word_to_num(corrected_potential_word)
    except ValueError:
        return potential_number_word
    return potential_num

In [32]:
def formatSexAge(origin_str):
    if not isinstance(origin_str, str):
        return [None,None]
    # Preprocess 
    # Entry with missing column.
    if (',' not in origin_str):
        origin_str = origin_str + ',';
    origin_list = origin_str.replace(" ", "").upper().split(",")
    if(origin_list[0].upper() == "NAN"):
        origin_list[0] = ""
    if(origin_list[1].upper() == "NAN"):
        origin_list[1] = ""
    # Convert possible number in entry.
    if((not is_number(origin_list[0])) and (not is_number(origin_list[1]))):
        origin_list[0] = numberConversion(origin_list[0])
        origin_list[1] = numberConversion(origin_list[1])
    genderSet = set(['F', 'M', 'OTHER'])
    if (is_number(origin_list[0])):
        # wrong entry (num, num)
        if (is_number(origin_list[1])):
            if (origin_list[0] == origin_list[1]):
                return [None, origin_list[0]]
            return [None, None]
        else: # first number, second '' or gender (NOT num for sure)
          # swap back number
          origin_list = origin_list[::-1]
          origin_list[0] = genderSpellingRewrite(origin_list[0])
          return origin_list
    else: 
        origin_list[0] = genderSpellingRewrite(origin_list[0])
         # first '' or str, second is number
        if (is_number(origin_list[1])):
            return origin_list
        else:
            origin_list[1] = genderSpellingRewrite(origin_list[1])
            if(origin_list[0] == origin_list[1]):
               origin_list[1] = None
            return origin_list

In [33]:
list(all_data)

['BMI',
 'TreatmentA',
 'TreatmentB',
 'TreatmentC',
 'TreatmentD',
 'average_blood_sugar',
 'heart_condition_detected_2017',
 'high_BP',
 'id',
 'married',
 'sex and age',
 'smoker_status',
 'job_status',
 'living_area']

In [34]:
def process_sex_age_(df):
    df["sex_age_list"] = df["sex and age"].astype(str).apply(lambda x: formatSexAge(x))
    df[['sex','age']] = pd.DataFrame(df["sex_age_list"].values.tolist(), index= df.index)
    df[df["sex"] == "None"]["sex"] = "OTHER"
process_sex_age_(all_data)

211      None
212      None
20916    None
20917    None
27514    None
36287    None
36877    None
36881    None
37127    None
37567    None
38047    None
39782    None
40067    None
41445    None
43207    None
Name: sex, dtype: object

Clean Sex and Age

In [35]:
def clean_sex_age(sex_age_list):
    if type(sex_age_list) is not list:
        return [None,None]
    # Strip and Upper case both sex and age
    sex_age_list[0],sex_age_list[1] = sex_age_list[0].strip().upper(), sex_age_list[1].strip().upper()
    
    # 2nd : first one is empty and second one is not numeric
    if (is_number(sex_age_list[0]) or (not sex_age_list[0] and not is_number(sex_age_list[1]))): 
        sex_age_list = sex_age_list[::-1]
    sex = sex_age_list[0].strip().upper()
    
    if sex in ('FEMALE','FEMALLE'):
        sex = 'F'
    if sex in ('MALE','MMALE','MM'):
        sex = 'M'

    sex_age_list[0] = sex
    sex_age_list[1] = sex_age_list[1]
    return sex_age_list

def process_sex_age(df):
    df["sex_age_list"] = df["sex and age"].str.split(",").apply(clean_sex_age)
    df[['sex','age']] = pd.DataFrame(df["sex_age_list"].values.tolist(), index= df.index)
    df["age"] = pd.to_numeric(df["age"],errors='coerce').round()
    return df.drop(columns=["sex_age_list","sex and age"])
all_data = process_sex_age(all_data)

In [36]:
def convert_to_categorical(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)
convert_to_categorical(all_data,["heart_condition_detected_2017","married","high_BP","job_status",
                                 "sex","smoker_status","living_area"])

In [550]:
#all_data.drop_duplicates(inplace=True)

In [551]:
def split_train_test(df):
    return df[:ntrain], df[ntrain:]

In [552]:
preprocessed_train, preprocessed_test = split_train_test(all_data)

### Missing Values

In [553]:
def impute_by_mode(df,columns):
    for col in columns:
        df[col] = df[col].fillna(all_data[col].mode().iloc[0])

In [554]:
impute_by_mode(all_data,["heart_condition_detected_2017","high_BP","married"])

In [555]:
all_data["sex"] = all_data["sex"].fillna("OTHER")

Check missing value of job_status

In [556]:
all_data.loc[all_data["id"] == 39698,"age"] = 20 # TODO Replace Age in preprocess

In [557]:
preprocessed_train, preprocessed_test = split_train_test(all_data)

In [558]:
all_data[all_data["BMI"].isnull()]

,BMI,TreatmentA,TreatmentB,TreatmentC,TreatmentD,average_blood_sugar,heart_condition_detected_2017,high_BP,id,married,smoker_status,job_status,living_area,sex,age
87,NaN,NaN,NaN,NaN,NaN,214.18,0.0,1.0,113,1.0,non-smoker,business_owner,city,F,79.0
179,NaN,NaN,NaN,NaN,NaN,188.75,0.0,0.0,231,1.0,None,government,city,M,64.0
181,NaN,NaN,NaN,NaN,NaN,203.29,0.0,1.0,233,1.0,active_smoker,private_sector,remote,M,57.0
211,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,290,nan,non-smoker,None,None,None,NaN
212,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,292,nan,None,None,None,None,NaN
300,NaN,NaN,NaN,NaN,NaN,80.75,0.0,0.0,407,1.0,None,private_sector,remote,F,62.0
316,NaN,0.0,0.0,0.0,0,92.87,1.0,0.0,432,1.0,non-smoker,private_sector,city,M,64.0
324,NaN,NaN,NaN,NaN,NaN,96.71,0.0,1.0,441,1.0,non-smoker,government,city,M,34.0
360,NaN,NaN,NaN,NaN,NaN,112.02,0.0,0.0,490,1.0,non-smoker,government,remote,F,46.0
470,NaN,NaN,NaN,NaN,NaN,231.30,0.0,0.0,628,1.0,active_smoker,private_sector,city,M,55.0


In [559]:
all_data[all_data["job_status"].isnull()]

,BMI,TreatmentA,TreatmentB,TreatmentC,TreatmentD,average_blood_sugar,heart_condition_detected_2017,high_BP,id,married,smoker_status,job_status,living_area,sex,age


In [560]:
preprocessed_train["job_status"].dtype

dtype('O')

In [591]:
def rf_predict_missing(df,missing_var,independent_var):
    non_missing_data = preprocessed_train[preprocessed_train[missing_var].notnull()]
    
    # Remove missing values
    non_missing_data.dropna(inplace=True)
    
    # Build Random Forest classifier
    clf = make_pipeline(OneHotEncoder(handle_unknown="ignore"),MultinomialNB())
    clf.fit(non_missing_data[independent_var],non_missing_data[missing_var])
    acc = np.mean(cross_val_score(clf,non_missing_data[independent_var],non_missing_data[missing_var],cv=5))
    print("Random Forest Mean Accuracy for 5 runs of cross validation ", acc)
    
    # Predict missing values
#     cond = df[missing_var].isnull()
#     df[cond][missing_var] = clf.predict(df[cond][independent_var])
    return clf

In [592]:
clf = rf_predict_missing(all_data,"job_status",["BMI","age","sex","married","high_BP"])

Random Forest Mean Accuracy for 5 runs of cross validation  0.5156094903774033


E:\Program\Anaconda\envs\NLP\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [ ]:
missing_var = 'job_status'
independent_var = ["age","sex","married","high_BP","heart_condition_detected_2017"]
non_missing_data = preprocessed_train[preprocessed_train[missing_var].notnull()][independent_var]
non_missing_data.dropna(inplace=True)
non_missing_data[non_missing_data.isna().any(axis=1)]

In [37]:
profile = all_data.profile_report(title='Medical Record Profiling Report')
profile.to_file(output_file="train_data_summary.html")